In [52]:
import numpy as np
import cv2

In [53]:
#image = cv2.imread('./left_image_74.png')
image = cv2.imread('./right_image_115.png')
if image is None:
    print("Erro ao carregar a imagem.")

In [54]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [55]:
#filter right camera

# lower_red1 = np.array([0, 100, 100])
# upper_red1 = np.array([10, 255, 255])
# mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)

lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([180, 255, 255])
mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

lower_red3 = np.array([170, 50, 50])
upper_red3 = np.array([180, 255, 255])
mask3 = cv2.inRange(hsv_image, lower_red3, upper_red3)

mask =  mask2 | mask3

In [56]:
# #filter left camera


# # lower_red1 = np.array([0, 100, 100])
# # upper_red1 = np.array([10, 255, 255])
# # mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)

# lower_red2 = np.array([160, 100, 100])
# upper_red2 = np.array([180, 255, 255])
# mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

# lower_red3 = np.array([170, 50, 30])
# upper_red3 = np.array([180, 255, 255])
# mask3 = cv2.inRange(hsv_image, lower_red3, upper_red3)

# mask =  mask3

In [57]:
blurred = cv2.GaussianBlur(mask, (9, 9), 10)

In [58]:
# Detectar círculos usando a Transformada de Hough
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                            param1=100, param2=20, minRadius=10, maxRadius=50)

In [59]:
rects = []

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        start_x, start_y = i[0] - i[2], i[1] - i[2]
        end_x, end_y = start_x + 2 * i[2], start_y + 2 * i[2]
        cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        rects.append(np.array((start_x, start_y, end_x, end_y), dtype="int"))

In [60]:
objects = None

In [61]:
def camera(self, camera_name):
    if camera_name == "left":
        objects = self.left_tracker.update(rects)
    elif camera_name == "right":
        objects = self.right_tracker.update(rects)

In [62]:
if objects:
    for objectID, centroid in objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(
            image,
            text,
            (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2,
        )
        cv2.circle(image, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

In [63]:
def kmeans_segmentation(
    self,
    image,
    attempts=10,
    k=4,
    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0),
):
    td_img = np.float32(image.reshape((-1, 3)))
    _, label, center = cv2.kmeans(
        td_img, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS
    )

    center = np.uint8(center)
    res = center[label.flatten()]
    
    return res.reshape((image.shape))


In [64]:
# def kmeans_segmentation(image, attempts=10, k=4, criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)):
#     td_img = np.float32(image.reshape((-1, 3)))
#     _, label, center = cv2.kmeans(td_img, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)

#     center = np.uint8(center)
#     res = center[label.flatten()]

#     return res.reshape((image.shape))

In [65]:
#kmeans = kmeans_segmentation(hsv_image)

In [66]:
# if circles is not None:
#         circles = np.uint16(np.around(circles))
#         for i in circles[0, :]:
#             # Desenhar o círculo na imagem original
#             cv2.circle(image, (i[0], i[1]), i[2], (0, 255, 0), 2)
#             # Desenhar o centro do círculo
#             cv2.circle(image, (i[0], i[1]), 2, (0, 0, 255), 3)

In [67]:
# # Contar o número de tomates detectados
# tomato_count = len(circles[0]) if circles is not None else 0
# print(f"Número de tomates detectados: {tomato_count}")

In [68]:
cv2.imshow('Imagem com Tomates Detectados', image)
cv2.imshow('HSV_image', hsv_image)
cv2.imshow('Máscara', mask)
cv2.imshow('blurred_image', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()